# 4.6 Secondary metabolite pathway addition

EcN produces six different iron-chelating siderophores;
- Aerobactin _(iuc/aer)_
- Enterobactin _(ent)_
- Salmochelin _(iro)_
- Yersiniabactin _(ybt)_
- Hemin-dependent iron acquisition system _(chu)_
- Citrate-dependent iron acquisition system _(cit)_

The latter two are already present in the model. The first four were added here. Colibactin _(clb/pks)_ was additionally added to the model.  

In [1]:
#import package needed
import cobra
import numpy as np
import pandas as pd
import re
from cobra.io import load_json_model
from glob import glob
from cobra import Model, Reaction, Metabolite

In [2]:
# Load EcN model
EcN_ID = 'CP022686.1'
EcN_model = cobra.io.load_json_model('../data/models/%s_cur_4.5.json'%EcN_ID)

In [3]:
# Load the tables with information on origin of genes & reactions
gene_origin_df = pd.read_csv('../tables/gene_origin.csv')
gene_origin_df.set_index('gene', inplace=True)

rxn_origin_df = pd.read_csv('../tables/rxn_origin.csv')
rxn_origin_df.set_index('reaction', inplace=True)

In [4]:
# rxn_origin_df = rxn_origin_df[rxn_origin_df['added'] != 'manual']

# 1. Add pathways

In [5]:
# First remove the ENTCS reaction, which is a lumped reaction of the enterobactin pathway
EcN_model.remove_reactions(['ENTCS'])

In [6]:
# Load the table with information on reactions to add
reaction_df = pd.read_csv('../tables/sec_met_paths_reactions.csv', sep=';') #adjust delimiter if needed
reaction_df['ID'] = reaction_df['ID'].str.strip() #Remove white spaces
reaction_df.set_index('ID', inplace=True)
reaction_df.head()

,Status,Sec_met_path,Name,Reaction Formula,Gene Rules,Gene Name,Subsystem,Compartment,LB,UB,RHEA,BioCyc,MetaNetX (MNX) Equation,KEGG Reaction,SEED Reaction,EC Number,Reactome Reaction,Reference PMID
ID,,,,,,,,,,,,,,,,,,
ICHORS,Draft model,Enterobactin,Isochorismate synthase,chor_c <=> ichor_c,CIW80_05275 or CIW80_20975,menF or entC,Secondary metabolite biosynthesis,c,-1000.0,1000.0,NaN,META:ISOCHORSYN-RXN,NaN,NaN,NaN,NaN,NaN,NaN
ICHORT,Draft model,Enterobactin,Isochorismatase,h2o_c + ichor_c --> 23ddhb_c + pyr_c,CIW80_20985,entB,Secondary metabolite biosynthesis,c,0.0,1000.0,NaN,META:ISOCHORMAT-RXN,NaN,NaN,NaN,NaN,NaN,NaN
DHBD,Draft model,Enterobactin,"2,3-dihydro-2,3-dihydroxybenzoate dehydrogenase",23ddhb_c + nad_c --> 23dhb_c + h_c + nadh_c,CIW80_20990,entA,Secondary metabolite biosynthesis,c,0.0,1000.0,NaN,META:DHBDEHYD-RXN,NaN,NaN,NaN,NaN,NaN,NaN
ArCPSentB,New Reaction,Enterobactin,Phosphopantetheinyl transferase (enterobactin B),apoArCPentB_c + coa_c --> holoArCPentB_c + h_c...,CIW80_20985 and CIW80_20915,entB and entD,Secondary metabolite biosynthesis,c,0.0,1000.0,NaN,META:ENTDB-RXN,MNXR115861,NaN,NaN,2.7.8.7,NaN,NaN
PCPSentF,New Reaction,Enterobactin,Phosphopantetheinyl transferase (enterobactin F),apoPCPentF_c + coa_c --> holoPCPentF_c + h_c +...,CIW80_20935 and CIW80_20915,entF and entD,Secondary metabolite biosynthesis,c,0.0,1000.0,NaN,META:RXN-15889,MNXR119687,NaN,NaN,2.7.8.7,NaN,NaN


In [7]:
reaction_df.rename(columns={'RHEA': 'rhea',
                            'BioCyc':'biocyc',
                            'MetaNetX (MNX) Equation': 'metanetx.reaction',
                            'KEGG Reaction': 'kegg.reaction',
                            'SEED Reaction': 'seed.reaction',
                            'EC Number': 'ec-code',
                            'Reactome Reaction': 'reactome',
                            'Reference PMID': 'pubmed'},
                    inplace = True)

reaction_df

,Status,Sec_met_path,Name,Reaction Formula,Gene Rules,Gene Name,Subsystem,Compartment,LB,UB,rhea,biocyc,metanetx.reaction,kegg.reaction,seed.reaction,ec-code,reactome,pubmed
ID,,,,,,,,,,,,,,,,,,
ICHORS,Draft model,Enterobactin,Isochorismate synthase,chor_c <=> ichor_c,CIW80_05275 or CIW80_20975,menF or entC,Secondary metabolite biosynthesis,c,-1000.0,1000.0,NaN,META:ISOCHORSYN-RXN,NaN,NaN,NaN,NaN,NaN,NaN
ICHORT,Draft model,Enterobactin,Isochorismatase,h2o_c + ichor_c --> 23ddhb_c + pyr_c,CIW80_20985,entB,Secondary metabolite biosynthesis,c,0.0,1000.0,NaN,META:ISOCHORMAT-RXN,NaN,NaN,NaN,NaN,NaN,NaN
DHBD,Draft model,Enterobactin,"2,3-dihydro-2,3-dihydroxybenzoate dehydrogenase",23ddhb_c + nad_c --> 23dhb_c + h_c + nadh_c,CIW80_20990,entA,Secondary metabolite biosynthesis,c,0.0,1000.0,NaN,META:DHBDEHYD-RXN,NaN,NaN,NaN,NaN,NaN,NaN
ArCPSentB,New Reaction,Enterobactin,Phosphopantetheinyl transferase (enterobactin B),apoArCPentB_c + coa_c --> holoArCPentB_c + h_c...,CIW80_20985 and CIW80_20915,entB and entD,Secondary metabolite biosynthesis,c,0.0,1000.0,NaN,META:ENTDB-RXN,MNXR115861,NaN,NaN,2.7.8.7,NaN,NaN
PCPSentF,New Reaction,Enterobactin,Phosphopantetheinyl transferase (enterobactin F),apoPCPentF_c + coa_c --> holoPCPentF_c + h_c +...,CIW80_20935 and CIW80_20915,entF and entD,Secondary metabolite biosynthesis,c,0.0,1000.0,NaN,META:RXN-15889,MNXR119687,NaN,NaN,2.7.8.7,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CLBTinter1,New Reaction,Colibactin,Colinactin intermediate reaction 1,preclbInter_p --> preclb16a_p + 4.0 h2o_p + 2....,NaN,NaN,Secondary metabolite biosynthesis,p,0.0,1000.0,NaN,RXN-21153,NaN,NaN,NaN,NaN,NaN,NaN
CLBTinter2,New Reaction,Colibactin,Colinactin intermediate reaction 2,preclb16a_p --> preclb16b_p + h_p,NaN,NaN,Secondary metabolite biosynthesis,p,0.0,1000.0,NaN,RXN-21102,NaN,NaN,NaN,NaN,NaN,NaN
CLBTinter3,New Reaction,Colibactin,Colinactin intermediate reaction 3,preclb16b_p + h2o_p + h_p --> clbtn_p + nh4_p,NaN,NaN,Secondary metabolite biosynthesis,p,0.0,1000.0,NaN,RXN-21101,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# add reactions
for rxn in reaction_df.index.values:
    
    # Create reaction and add to model, add reaction name and subsystem
    reaction = cobra.Reaction(rxn)
    if reaction not in EcN_model.reactions: 
        print('>>>> ADDING', rxn) 
        EcN_model.add_reaction(reaction)

        # Add name and subsystem
        reaction.name = reaction_df.loc[rxn, 'Name']
        reaction.subsystem = reaction_df.loc[rxn, 'Subsystem']

        # Add boundaries when given
        if pd.isnull(reaction_df.loc[rxn, 'LB']):
            pass
        else:
            reaction.lower_bound = reaction_df.loc[rxn, 'LB']
        if pd.isnull(reaction_df.loc[rxn, 'UB']):
            pass
        else:
            reaction.upper_bound = reaction_df.loc[rxn, 'UB']

        # Add reaction formula
        reaction.build_reaction_from_string(reaction_df.loc[rxn, 'Reaction Formula'])

        # Add gene reaction rule when given
        if str(reaction_df.loc[rxn, 'Gene Rules']).find('CIW80') != -1: # If "find" doesn't find a match, it returns -1
            reaction.gene_reaction_rule = reaction_df.loc[rxn, 'Gene Rules']

        # Add compartment
        if 'and' in reaction_df.loc[rxn, 'Compartment']: # take into account the cases where there are two comparments
            compartment = reaction_df.loc[rxn, 'Compartment'].split()
            reaction.compartments.add(compartment[0]) # add the compartment in front of "and"
            reaction.compartments.add(compartment[2]) # add the compartment behind "and"
            if len(compartment) == 5:
                reaction.compartments.add(compartment[4]) # add the compartment behind "and" 
        else:
            reaction.compartments.add(reaction_df.loc[rxn, 'Compartment'])

        # Add database annotations when present
        for annot in ['rhea', 'biocyc','metanetx.reaction', 'kegg.reaction', 'seed.reaction', 'ec-code', 'reactome', 'pubmed']:
            if pd.isnull(reaction_df.loc[rxn, annot]):
                pass
            else:
                reaction.annotation[annot] = [reaction_df.loc[rxn, annot]]

        # Add to reaction overview
        rxn_origin_df.loc[rxn, 'origin'] = reaction_df.loc[rxn, 'Status']
        rxn_origin_df.loc[rxn, 'added'] = 'manual'
        rxn_origin_df.loc[rxn, 'notebook'] = '4.6'
        rxn_origin_df.loc[rxn, 'notes'] = 'none'


        ### Genes ###

        # Update gene name
        gene_name = re.split(' and | or ', str(reaction_df.loc[rxn, 'Gene Name']))
        gene_rules = re.split(' and | or ', str(reaction_df.loc[rxn, 'Gene Rules']))

        if len(gene_name) == len(gene_rules) and gene_rules[0].find('CIW80') != -1: # If "find" doesn't find a match, it returns -1
            for idx, gene in enumerate(gene_rules):
                EcN_model.genes.get_by_id(gene).name = gene_name[idx]

                # Register the genes
                gene_origin_df.loc[gene, 'EcN_gene'] = gene
                gene_origin_df.loc[gene, 'origin'] = reaction_df.loc[rxn, 'Status']
                gene_origin_df.loc[gene, 'added'] = 'manual'
                gene_origin_df.loc[gene, 'notebook'] = '4.6'
                gene_origin_df.loc[gene, 'notes'] = 'none'
            
    ### If reaction is present, update information
    else:
        print('>>>> UPDATING', rxn) 
        EcN_model.reactions.get_by_id(rxn)
        # Add name and subsystem
        EcN_model.reactions.get_by_id(rxn).name = reaction_df.loc[rxn, 'Name']
        EcN_model.reactions.get_by_id(rxn).subsystem = reaction_df.loc[rxn, 'Subsystem']

        # Add boundaries when given
        if pd.isnull(reaction_df.loc[rxn, 'LB']):
            pass
        else:
            EcN_model.reactions.get_by_id(rxn).lower_bound = reaction_df.loc[rxn, 'LB']
        if pd.isnull(reaction_df.loc[rxn, 'UB']):
            pass
        else:
            EcN_model.reactions.get_by_id(rxn).upper_bound = reaction_df.loc[rxn, 'UB']

        # Add reaction formula
        EcN_model.reactions.get_by_id(rxn).build_reaction_from_string(reaction_df.loc[rxn, 'Reaction Formula'])

        # Add gene reaction rule when given
        if str(reaction_df.loc[rxn, 'Gene Rules']).find('CIW80') != -1: # If "find" doesn't find a match, it returns -1
            EcN_model.reactions.get_by_id(rxn).gene_reaction_rule = reaction_df.loc[rxn, 'Gene Rules']

        # Add compartment
        if 'and' in reaction_df.loc[rxn, 'Compartment']: # take into account the cases where there are two comparments
            compartment = reaction_df.loc[rxn, 'Compartment'].split()
            EcN_model.reactions.get_by_id(rxn).compartments.add(compartment[0]) # add the compartment in front of "and"
            EcN_model.reactions.get_by_id(rxn).compartments.add(compartment[2]) # add the compartment behind "and"
            if len(compartment) == 5:
                EcN_model.reactions.get_by_id(rxn).compartments.add(compartment[4]) # add the compartment behind "and" 
        else:
            EcN_model.reactions.get_by_id(rxn).compartments.add(reaction_df.loc[rxn, 'Compartment'])

        # Add database annotations when present
        for annot in ['rhea', 'biocyc','metanetx.reaction', 'kegg.reaction', 'seed.reaction', 'ec-code', 'reactome', 'pubmed']:
            if pd.isnull(reaction_df.loc[rxn, annot]):
                pass
            else:
                EcN_model.reactions.get_by_id(rxn).annotation[annot] = [reaction_df.loc[rxn, annot]]

>>>> UPDATING ICHORS
>>>> UPDATING ICHORT
>>>> UPDATING DHBD
>>>> ADDING ArCPSentB
unknown metabolite 'apoArCPentB_c' created
unknown metabolite 'holoArCPentB_c' created
>>>> ADDING PCPSentF
unknown metabolite 'apoPCPentF_c' created
unknown metabolite 'holoPCPentF_c' created
>>>> ADDING ArCPpdsentB
>>>> ADDING PCPpdsentF
>>>> UPDATING DHBS
>>>> UPDATING SERASr
>>>> ADDING ENTERS1
unknown metabolite '23dhbaArCPentB_c' created
>>>> ADDING ENTERS2
unknown metabolite 'serPCPentF_c' created
>>>> ADDING ENTERS3
unknown metabolite 'enter1PCPentF_c' created
>>>> ADDING ENTERS4
>>>> UPDATING ENTERES
>>>> UPDATING FEENTERtpp
>>>> UPDATING FEENTER2tpp
>>>> UPDATING FEENTERtex
>>>> UPDATING FEENTERexs
>>>> UPDATING FEENTERtonex
>>>> UPDATING FEENTERabcpp
>>>> UPDATING FEENTERR1
>>>> UPDATING FEENTERR2
>>>> UPDATING FEENTERR4
>>>> UPDATING FEENTERR3
>>>> UPDATING FE3DHBZSabcpp
>>>> UPDATING DHBSZ3FEabcpp
>>>> UPDATING FEENTERES
>>>> UPDATING FE3DHBZS3R
>>>> UPDATING ENTERES2
>>>> UPDATING EX_feente

unknown metabolite 'preclbInter_p' created
unknown metabolite 'myrasp_D_p' created
>>>> ADDING DM_MYRASP
>>>> ADDING CLBTinter1
unknown metabolite 'preclb16a_p' created
>>>> ADDING CLBTinter2
unknown metabolite 'preclb16b_p' created
>>>> ADDING CLBTinter3
unknown metabolite 'clbtn_p' created
>>>> ADDING CLBTNtex
unknown metabolite 'clbtn_e' created
>>>> ADDING EX_clbtn_e


In [9]:
# Check correct addition of genes to gene overview table
gene_origin_df

,EcN_gene,origin,added,notebook,notes,gene.1,gene.1.1,gene.1.1.1,gene.1.1.1.1
gene,,,,,,,,,
b2551,CIW80_06560,iML1515,automatic,3.1,none,NaN,NaN,NaN,NaN
b0870,CIW80_22360,iML1515,automatic,3.1,none,NaN,NaN,NaN,NaN
b3368,CIW80_11380,iML1515,automatic,3.1,none,NaN,NaN,NaN,NaN
b2436,CIW80_06055,iML1515,automatic,3.1,none,NaN,NaN,NaN,NaN
b0008,CIW80_17715,iML1515,automatic,3.1,none,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
CIW80_03455,CIW80_03455,New Reaction,manual,4.6,none,NaN,NaN,NaN,NaN
CIW80_03445,CIW80_03445,New Reaction,manual,4.6,none,NaN,NaN,NaN,NaN
CIW80_03440,CIW80_03440,New Reaction,manual,4.6,none,NaN,NaN,NaN,NaN


# 2. Update metabolites

In [10]:
# Load the table with information on reactions to add
mtb_df = pd.read_csv('../tables/sec_met_paths_metabolites.csv')
mtb_df['metabolite_ID'] = mtb_df['metabolite_ID'].str.strip() #Remove white spaces
mtb_df.set_index('metabolite_ID', inplace=True)

# Names in the csv file are already in the correct annotation format
mtb_df

,Status,metabolite_name,formula,compartment,formula_weight,charge,bigg.metabolite,inchikey,metanetx.chemical,sbo,seed.compound,biocyc,chebi,envipath,hmdb,kegg.compound,kegg.drug,reactome.compound,sabiork.compound
metabolite_ID,,,,,,,,,,,,,,,,,,,
holoArCPentB_c,New metabolite,Holo-[EntB aryl-carrier protein],C14H25N3O8PSR,c,NaN,-1.0,NaN,NaN,MNXM147417,NaN,NaN,META:Holo-EntB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
apoArCPentB_c,New metabolite,Apo-[EntB aryl-carrier protein],C3H5NO2R,c,NaN,0.0,NaN,NaN,MNXM148907,NaN,NaN,META:Apo-EntB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
holoPCPentF_c,New metabolite,Holo-[EntF L-seryl-carrier protein],C14H25N3O8PSR,c,NaN,-1.0,NaN,NaN,MNXM147418,NaN,NaN,META:Holo-EntF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
apoPCPentF_c,New metabolite,Apo-[EntF L-seryl-carrier protein],C3H5NO2R,c,NaN,0.0,NaN,NaN,MNXM148908,NaN,NaN,META:Apo-EntF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23dhbaArCPentB_c,New metabolite,"2,3-dihydroxybenzoyl-[EntB aryl-carrier protein]",C21H29N3O11PSR,c,NaN,-1.0,NaN,NaN,MNXM147916,NaN,NaN,META:23DHB-EntB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
preclbInter_p,New metabolite,unstable precolibactin intermediate,C37H52N9O10S2,p,843.30438,3.0,NaN,GYHFSHJPDFYZDE-SHROTLKGSA-Q,NaN,NaN,NaN,META:CPD-22758,NaN,NaN,NaN,NaN,NaN,NaN,NaN
preclb16a_p,New metabolite,precolibactin 16a,C37H42N9O6S2,p,771.26212,1.0,NaN,JWSSANKUQQCBBA-UWGAKSRVSA-O,NaN,NaN,NaN,META:CPD-22721,NaN,NaN,NaN,NaN,NaN,NaN,NaN
preclb16b_p,New metabolite,precolibactin 16b,C37H39N9O6S2,p,769.24647,0.0,NaN,NWUPXZAZUAPYEW-ROUUACIJSA-N,NaN,NaN,NaN,META:CPD-22720,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# update metabolites
for mtb in mtb_df.index.values:
    print('>>>> UPDATING', mtb)
    
    # Get the metabolite
    mtb = EcN_model.metabolites.get_by_id(mtb)
    
    # Update name, formula, compartment & charge, when this is not yet present
    if len(mtb.name) == 0:
        mtb.name = mtb_df.loc[mtb.id, 'metabolite_name']
#         mtb.formula = mtb_df.loc[mtb.id, 'formula']
        mtb.compartment = mtb_df.loc[mtb.id, 'compartment']
        
    if pd.isnull(mtb_df.loc[mtb.id, 'charge']):
        print('No charge')
    else:
        mtb.charge = int(mtb_df.loc[mtb.id, 'charge'])

    

    # Add database annotations when present
    for annot in ['bigg.metabolite', 'inchikey', 'metanetx.chemical', 'sbo', 'seed.compound', 'biocyc', 'chebi',
                  'envipath', 'hmdb', 'kegg.compound', 'kegg.drug', 'reactome.compound', 'sabiork.compound']:
        if pd.isnull(mtb_df.loc[mtb.id, annot]):
            pass
        else:
            mtb.annotation[annot] = [mtb_df.loc[mtb.id, annot]]

>>>> UPDATING holoArCPentB_c
>>>> UPDATING apoArCPentB_c
>>>> UPDATING holoPCPentF_c
>>>> UPDATING apoPCPentF_c
>>>> UPDATING 23dhbaArCPentB_c
>>>> UPDATING serPCPentF_c
>>>> UPDATING enter1PCPentF_c
>>>> UPDATING entertg_c
>>>> UPDATING n6hlys_c
>>>> UPDATING n6acn6hlys_c
>>>> UPDATING n2citn6acn6hlys_c
>>>> UPDATING apoACPirp1_c
>>>> UPDATING apoArCPirp2_c
>>>> UPDATING apoPCP1irp2_c
>>>> UPDATING apoPCP2irp2_c
>>>> UPDATING apoPCP3irp1_c
>>>> UPDATING holoACPirp1_c
>>>> UPDATING holoArCPirp2_c
>>>> UPDATING holoPCP1irp2_c
>>>> UPDATING holoPCP2irp2_c
>>>> UPDATING holoPCP3irp1_c
>>>> UPDATING salamp_c
>>>> UPDATING cysamp_c
>>>> UPDATING salArCPirp2_c
>>>> UPDATING cysPCP1irp2_c
>>>> UPDATING salcysPCP1irp2_c
No charge
>>>> UPDATING hptPCP1irp2_c
No charge
>>>> UPDATING cysPCP2irp2_c
No charge
>>>> UPDATING hptcysPCP2irp2_c
No charge
>>>> UPDATING hpttPCP2irp2_c
No charge
>>>> UPDATING malACPirp1_c
No charge
>>>> UPDATING KSirp1_c
No charge
>>>> UPDATING hpttKSirp1_c
No charge
>>>> 

In [12]:
EcN_model.reactions.FEYBTR.check_mass_balance()

{'charge': 3, 'Fe': 1}

# 3. Check pathways

In [13]:
with EcN_model:
    for rxn in ['EX_arbtn_e', 'EX_feenter_e', 'EX_enter_e', 'EX_salchsx_e', 'EX_ybt_e', 'EX_feybt_e', 'EX_znybt_e', 'EX_cuybt_e', 'EX_niybt_e', 'CLBTNtex']:
#     for rxn in reaction_df.index.values:
        EcN_model.objective = EcN_model.reactions.get_by_id(rxn)
        solution = EcN_model.slim_optimize()
        print(rxn, solution)

EX_arbtn_e 4.513220338983051
EX_feenter_e 3.0581974248927075
EX_enter_e 3.0581974248927075
EX_salchsx_e 5.7647586451079915
EX_ybt_e 1.983303556927076
EX_feybt_e 1.9833035569270747
EX_znybt_e 1.9833035569270752
EX_cuybt_e 1.9833035569270752
EX_niybt_e 1.9833035569270752
CLBTNtex 1.0111711711711728


In [14]:
rxn_origin_df

,origin,added,notebook,notes
reaction,,,,
CYTDK2,iML1515,automatic,3.1,none
XPPT,iML1515,automatic,3.1,none
HXPRT,iML1515,automatic,3.1,none
NDPK5,iML1515,automatic,3.1,none
SHK3Dr,iML1515,automatic,3.1,none
...,...,...,...,...
CLBTinter1,New Reaction,manual,4.6,none
CLBTinter2,New Reaction,manual,4.6,none
CLBTinter3,New Reaction,manual,4.6,none


# 4. Methionine salvage pathway
In the paper by Sun et al. (2015), they describe two reactions in which EcN converts methylthioribose (MTR) to methylthioribulose-1-phosphate (MTRu-1-P) by two enzymes:
- MTR kinase (mtnK, ecn2295)
- methylthioribose-1-phosphate (MTR-1-P) isomerase (mtnA, ecn2294)

These reactions are present in the BiGG database as "MTRK" (5-methylthioribose kinase) and "MTRI" (5-methylthioribose-1-phosphate isomerase). Both are added here to the model.

Homology:
- ecn2295 > Q986M8 > CIW80_17025 (based on homology)
- ecn2294 > CIW80_17030 (based on annotation in genome: mtnA, "S-methyl-5-thioribose-1-phosphate isomerase")

In [15]:
### Added from iDK1463

# # Load reference model
# ref_model = cobra.io.load_json_model('../data/iAF1260.json')

# # Add reactions to EcN
# EcN_model.add_reactions([ref_model.reactions.MTRK, ref_model.reactions.MTRI])

# # Add genes
# EcN_model.reactions.MTRK.gene_reaction_rule = 'CIW80_17025'
# EcN_model.reactions.MTRI.gene_reaction_rule = 'CIW80_17030'

# # Add gene names
# EcN_model.genes.CIW80_17025.name = 'mtnK (methylthioribose (MTR) kinase)'
# EcN_model.genes.CIW80_17030.name = 'mtnA (methylthioribose-1-phosphate (MTR-1-P) isomerase)'

# for rxn in ['MTRK', 'MTRI']:
#     # Add to reaction overview
#     rxn_origin_df.loc[rxn, 'origin'] = 'iAF1260'
#     rxn_origin_df.loc[rxn, 'added'] = 'manual'
#     rxn_origin_df.loc[rxn, 'notebook'] = '4.6'
#     rxn_origin_df.loc[rxn, 'notes'] = 'Added based on Sun et al. (2005) in notebook 4.6'
        
# for gene in ['CIW80_17025', 'CIW80_17030']:
#     # Register the genes
#     gene_origin_df.loc[gene, 'EcN_gene'] = gene
#     gene_origin_df.loc[gene, 'origin'] = 'iAF1260'
#     gene_origin_df.loc[gene, 'added'] = 'manual'
#     gene_origin_df.loc[gene, 'notebook'] = '4.6'
#     gene_origin_df.loc[gene, 'notes'] = 'Added based on Sun et al. (2005) in notebook 4.6'

In [16]:
EcN_model.genes.CIW80_17025

Gene identifier,CIW80_17025
Name,S-methyl-5-thioribose kinase
Memory address,0x026f57b8ef48
Functional,True
In 1 reaction(s),MTRK


# Others
Overview from Sun et al. (2005) > Enzymes present in E. coli Nissle

- 2.3.1.12	Dihydrolipoamide acetyltransferase component (E2) of pyruvate dehydrogenase complex,	Glycolysis, gluconeogenesis and pyruvate metabolism > PDH, present
- 2.7.1.51	FucK; l-fuculokinase,	Fructose and mannose metabolism > FCLK, present
- 2.8.3.1	Propionate CoA-transferase,	Pyruvate and propanoate metabolism > PPCSCT, present
- 3.5.3.6	Arginine deiminase,	Arginine and proline metabolism > ARGDr, present
- 1.1.1.31	3-Hydroxyisobutyrate dehydrogenase,	Val, Leu and Ile degradation <font color=red> > HIBD, not present. Pathway not present in the model </font>
- 1.1.3.15	(S)-2-Hydroxy-acid oxidase,	Glyoxylate and dicarboxylate metabolism > <font color=red> no reaction on BiGG. </font>
- 2.7.1.53	l-Xylulokinase,	Pentose and glucuronate interconversions > XYLK/XYLK2, present
- 2.7.1.58	2-Dehydro-3-deoxygalactonokinase,	Galactose metabolism > DDGALK, present
- 2.8.3.8	Acetate CoA-transferase,	Propanoate and butanoate metabolism > ACACCT, present
- 3.2.1.14	Chitinase,	Aminosugars metabolism > <font color=red> no reaction on BiGG. </font>

Other reactions were not added

# 5. Save model

In [17]:
len(EcN_model.genes)

1536

In [18]:
# Save the model
cobra.io.json.save_json_model(EcN_model, str('../data/models/%s_cur_4.6.json'%EcN_ID))

In [19]:
# Save reaction and gene origin tables
rxn_origin_df.to_csv('../tables/rxn_origin.csv')
gene_origin_df.to_csv('../tables/gene_origin.csv')